<a href="https://colab.research.google.com/github/rafaborneo/graph_course/blob/main/Borneo_Graphs_Practico1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico 1

En este práctico, correspondiente a la primera mitad de la materia, trabajaremos con un dataset extraído de Twitter. La idea es emplear los conceptos de grafos vistos en clase sobre un caso real de actualidad.

## Dataset

El dataset consiste en un conjunto de hilos de tweets, con un total de ~150000 tweets, extraídos entre Enero y Marzo de 2021. La temática de los mismos está referida a la vacunación contra el covid-19 en Argentina.

Pueden descargar el dataset del siguiente [link](https://drive.google.com/file/d/1X_qKsE8muAnom2tDX4sLlmBAO0Ikfe_G/view?usp=sharing).

### Campos

- **created_at:** Fecha del tweet
- **id_str:** ID del tweet
- **full_text:** Contenido del tweet
- **in_reply_to_status_id:** ID del tweet inmediatamente anterior en el hilo
- **in_reply_to_user_id:** Autor del tweet inmediatamente anterior en el hilo
- **user.id:** Autor del tweet
- **user_retweeters:** Lista de ID de usuarios que retweetearon el tweet
- **sentiment:** Etiquetado manual que indica el sentimiento o intención del tweet con respecto al tweet anterior en el hilo

## Configuración inicial

In [11]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_context('talk')
#from pathlib import Path

## Descargar el csv con los datos en este directorio
#DATA_DIR = Path('C:\Users\Rafael\Documents\AREA TRABAJO\DIPLODATOS 2021\7_opt_Analisis_Grafos\clases\Practico')
#INPUT_FILE = DATA_DIR / 'vacunas.csv'

## Creamos el directorio en caso de que no exista
#DATA_DIR.mkdir(parents=True, exist_ok=True)

### Cargamos el dataset

In [12]:
dtypes = {
    'id_str': str,
    'full_text': str,
    'in_reply_to_status_id': str,
    'in_reply_to_user_id': str,
    'user.id': str
}
df = pd.read_csv("vacunas.csv", dtype=dtypes).dropna(subset=['user_retweeters'])
df['user_retweeters'] = df['user_retweeters'].apply(lambda x: [str(elem) for elem in eval(x)])
print(df.shape)
df.head()

(155123, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,"[2258074658, 159909978, 105301854, 290671142, ...",NaN
1,Sat Feb 20 03:19:59 +0000 2021,1362965193509265417,@Clon_43 @norabar Nora estaba indignada porque...,1362962469749153792,2737379453,32718111,[],NaN
2,Mon Feb 22 23:55:08 +0000 2021,1364000806740111363,"Bueno, Alberto dijo Salud o Economía. La salud...",NaN,NaN,252168075,"[1238117630696972289, 37232479, 12792246571247...",NaN
3,Tue Feb 23 00:09:14 +0000 2021,1364004354374696963,@spitta1969 Tuit del mes Spitta,1364000806740111363,252168075,1156346340802224128,[],NaN
4,Tue Feb 23 00:00:17 +0000 2021,1364002100364128260,@spitta1969 Estas onfire,1364000806740111363,252168075,153663816,[],NaN


### Observamos algunos ejemplos

In [13]:
idx = 0
print('Texto:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Texto: Seguimos esperando el comunicado de @norabar repudiando la situación respecto del gobierno y el tema vacunas. Seamos pacientes que con esto de la pandemia anda con mucho "laburo".
Retweets: 9


In [14]:
idx = 376
print('Text:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Text: Todo lo que hay que entender sobre la decisión –o no– de poner más vacunas en más brazos (por ejemplo, usar las 1º dosis en muchos y si es necesario retrasar la 2º) está en esta excelente nota de Nora Bär. https://t.co/A0I03DyxgO
Retweets: 48


### Calculamos la cantidad de hilos

In [15]:
roots = df[df['in_reply_to_user_id'].isna()]
roots.shape

(3174, 8)

## Actividades

### **1. Construcción del grafo** 

Construir el **grafo de retweets**, definido de la siguiente manera:

- Tipo de grafo: Dirigido
- Nodos: ID de los usuarios
- Enlaces: (Usuario A) ---> (Usuario B) si B retweeteó algún tweet de A

Con estos datos, el grafo debería tener alrededor de 40000 nodos y 90000 enlaces.

Considerar la versión no dirigida del grafo y estudiar su conectividad. Si existe una única "componente gigante", realizar el resto de las actividades sobre ella, en lugar de sobre el grafo completo.

Calcular las siguientes métricas globales del grafo:

- Grado medio
- Asortatividad
- Transitividad
- Coeficiente de clustering de Watts-Strogatz

**Opcional:** Comparar las métricas calculadas anteriormente con las de un grafo aleatorio con la misma distribución de grado. Pueden utilizar para ello este [método](https://networkx.org/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html?highlight=configuration#networkx.generators.degree_seq.configuration_model). Con esto en mente, comentar si los valores obtenidos anteriormente difieren significativamente del caso aleatorio.


### **2. Centralidad**

Calcular 5 métricas de centralidad de nodos. Graficar la distribución de cada una de ellas ¿Existe alguna correlación entre las distintas centralidades? 

Hacer un ranking con los 10 nodos más centrales para cada métrica. ¿Hay coincidencia entre los rankings?. ¿Qué características tienen los usuarios más centrales y sus respectivos tweets?

**Opcional:** Determinar si existe alguna correlación entre la centralidad de un nodo y su actividad en red social. Es decir, evaluar si los usuarios que más escriben son los más centrales o no.

### **3. Comunidades**

Utilizar el algoritmo de Louvain con el parámetro "resolución" igual a 1. Caracterizar las comunidades halladas (cantidad, distribución de tamaños). Utilizar la modularidad y otras dos métricas a elección para evaluar la calidad de la partición encontrada. 

Variar el parámetro "resolución" y observar cómo cambia la distribución de comunidades encontradas. ¿Existe algún valor para el cual se identifiquen dos grandes comunidades?

Elegir otro algoritmo de detección de comunidades y comparar los resultados con los obtenidos anteriormente.

**Opcional:** Correr el algoritmo de Louvain con distintas semillas aleatorias. Utilizar alguna métrica de comparación externa entre las particiones obtenidas para determinar en qué medida depende el algoritmo de la condición inicial.

In [16]:

#expansion columna user_retweeters
df1= df.explode("user_retweeters", ignore_index=True)
print(df1.shape)
df1.head()

(320948, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,2258074658,NaN
1,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,159909978,NaN
2,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,105301854,NaN
3,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,290671142,NaN
4,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,208721168,NaN


In [19]:
df1 = df1.dropna(subset=['user.id','user_retweeters'])
print(df1.shape)
df1.head()

(187297, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,2258074658,NaN
1,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,159909978,NaN
2,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,105301854,NaN
3,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,290671142,NaN
4,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,208721168,NaN


In [20]:
dir_G =  nx.from_pandas_edgelist(df1, source='user.id', target='user_retweeters', edge_attr=None, create_using=None, edge_key=None)

In [21]:
print(dir_G)

Graph with 39800 nodes and 93404 edges


In [ ]:
#METRICAS GLOBALES

In [22]:
print(nx.info(dir_G))

Graph with 39800 nodes and 93404 edges


In [37]:
#lista de grados del grafo
grados = [val for (node, val) in dir_G.degree()]
print(grados)

[9, 19, 1, 10, 15, 1, 8, 7, 22, 16, 8207, 3, 4, 5, 1, 11, 1, 4, 1, 1, 1, 2, 10, 1, 4, 1, 1, 1, 1, 13, 6, 1, 1, 2, 1, 2, 3, 1, 3, 1, 3, 4, 1, 7, 16, 1, 1, 91, 1, 26, 1, 12, 1, 19, 1, 8, 4, 34, 3, 9, 28, 8, 19, 47, 72, 186, 2, 97, 45, 43, 6, 11, 26, 4, 9, 36, 5, 8, 1, 3, 12, 7, 199, 7, 28, 5, 7, 5362, 14, 1, 2, 4, 2, 24, 3, 1, 7, 8, 1, 1, 17, 1, 1, 1, 2, 7, 14, 10, 19, 5, 28, 4, 10, 3, 35, 10, 20, 1, 7, 18, 1, 1, 1, 14, 118, 24, 12, 10, 23, 10, 7, 26, 2, 6, 2, 8, 13, 33, 7, 5, 13, 7, 26, 29, 13, 3, 5, 5, 112, 5, 10, 3, 48, 15, 15, 2, 19, 3, 1, 7, 3, 8, 3, 26, 14, 9, 6, 2, 13, 3, 24, 29, 11, 1, 2, 24, 10, 4, 10, 9, 129, 24, 1, 32, 6, 9, 13, 22, 6, 2, 30, 15, 1, 31, 10, 11, 14, 3, 61, 3, 4, 7, 3, 133, 26, 6, 6, 9, 67, 2, 1, 10, 95, 4, 1, 7, 5, 2, 1, 19, 1, 5, 2, 31, 3, 13, 20, 1, 4, 46, 12, 1, 10, 3, 3, 14, 4, 24, 8, 3, 21, 7, 1, 1, 12, 10, 8, 1, 6, 1, 4, 4, 19, 1, 12, 75, 13, 11, 2, 1, 11, 6, 4, 5, 12, 1, 11, 278, 1, 18, 10, 1, 8, 6, 8, 2, 3849, 2, 1, 4, 46, 1, 10, 3, 1, 163, 2, 2, 5, 5, 

In [36]:
#grado medio:promedio de los grados de cada nodo
grado_medio= sum(grados)/len(grados)
print(grado_medio)

4.693668341708543


In [39]:
#ASORTATIVIDAD BASADA EN EL GRADO; DEGREE CORRELATION; DEGREE ASSORTATIVITY
r = nx.degree_assortativity_coefficient(dir_G)
print(r)
#el valor negaticoindica  que la red es disortativa


-0.2213190254294167


In [43]:
#Construimos un DataFrame de pandas para almacenar la estadística de los nodos
df3 = pd.DataFrame(index=list(dir_G.nodes()))
df3.shape

(39800, 0)

# TRANSITIVIDAD Coeficiente de clustering 

In [46]:
clustering_coefficient = nx.clustering(dir_G)
df3['Cws'] = list(clustering_coefficient.values())
df3

,Cws
2737379453,0.000000
2258074658,0.052632
159909978,0.000000
105301854,0.066667
290671142,0.066667
...,...
2765081259,0.000000
339254354,0.000000
346605471,0.000000
58065646,0.000000
